In [1]:
inst <- suppressMessages(lapply(c("DESeq2",
                                  "DEP",
                                  "ggplot2",
                                  "gplots",
                                  "RColorBrewer",
                                  "clusterProfiler",
                                  "org.Mm.eg.db",
                                  "gridExtra",
                                  "readxl"), 
                                library,
                                character.only=TRUE)
)

# set the directory
dir_out = '~/Documents/consultation/Katarina/fasting_therapies/results'
dir_tab = file.path(dir_out, 'tables')
dir_fig = file.path(dir_out, 'figures')
dir_dat = file.path(dir_out, 'anndata')

# set the colors and options
colors = colorRampPalette(rev(brewer.pal(9, "Spectral")))(255)
my_palette = colorRampPalette(c("blue", "white", "red"))(n=255)
options(repr.plot.width=10, repr.plot.height=10)

Warning message in fun(libname, pkgname):
“mzR has been built against a different Rcpp version (1.0.4.6)
than is installed on your system (1.0.5). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue.”


### Set the global variables

Set whether anndata objects are recomputed or loaded from cache.

In [2]:
bool_recomp = TRUE

Set whether to produce plots, set to False for test runs.

In [3]:
bool_plot = FALSE

In [4]:
# set input file
setwd('~/Documents/consultation/Katarina/fasting_therapies/input')

# load the data
rna <- read.csv('rna_expression_data.csv',
                row.names=1
)
prot <- read.csv('if_study.csv', 
                 sep=';'
)

# Load functions
source("rna_prot_functions.R")

# Obsolete terms
noliv <- read.csv("nonliver_terms.csv",
                  header=FALSE
)
obsolete <- read.csv("obsolete_GO_terms.csv",
                     header=FALSE
)

## transcriptomics

In [5]:
# get samples of interest
ifrna = rna[,grep("IFHFD_10|ALHFD_10", colnames(rna))]

# make meta data object
id = colnames(ifrna)
condition = factor(gsub('.{3}$', '', colnames(ifrna)))
metaData = data.frame(id, condition)

# make deseq object
dds <- DESeqDataSetFromMatrix(countData=ifrna, 
                              colData=metaData, 
                              design=~condition
)  

# filter low count features
dds <- dds[rowSums(counts(dds)) >= 10,]

# differential gene expression
dds <- DESeq(dds)

# get the result of differntial expression analysis
dge <- results(dds)

# order and indicate significat features
res = dge[order(dge$pvalue),]
res_dge = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                      significant=ifelse(res$pvalue < 0.05, "pvalue<0.05", "not signif.")), 
                        row.names=rownames(res))

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [4]:
if(bool_plot){
    # volcano and PCA plot 
    plot_fig_rna(deg_table=res_dge,
                 dds_object=dds, 
                 col_volcano=c("grey", "firebrick"),
                 col_pca=c("firebrick4", "firebrick1"),
                 xlim=c(-5,5),
                 ylim=c(0,20)
    )
}

In [7]:
# convert ensembl name to gene name
ifd = res_dge[,c(2,5,6)]
ifd$ENSEMBL = rownames(ifd)
sig.gene <- bitr(ifd$ENSEMBL,
                 fromType="ENSEMBL",
                 toType="SYMBOL",
                 OrgDb=org.Mm.eg.db)
rnaIF <- merge(ifd,
               sig.gene,
               by="ENSEMBL"
)[,c(5,2,3)]

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(ifd$ENSEMBL, fromType = "ENSEMBL", toType = "SYMBOL", OrgDb = org.Mm.eg.db):
“23.61% of input gene IDs are fail to map...”


In [8]:
# save the normalized data
write.csv(round(counts(dds, normalized=TRUE), 0), 
          file.path(dir_dat, "normalized_data.csv"),
          row.names = TRUE
)

In [9]:
# variance stabilizing transformation data
vsd <- assay(vst(dds, 
                 fitType="local",
                 blind = TRUE
             )
        )

## proteomics

In [11]:
data_unique <- make_unique(prot,
                           "PG.Genes",
                           "PG.ProteinDescriptions", 
                           delim="/t"
)
a_columns <- grep("IFHFD|ALHFD",
                  colnames(data_unique))[-4] # -4 remove sample

label = colnames(data_unique)[a_columns]
condition = rep(c("IFHFD", "ALHFD"), 7) 

replicate = as.character(c(1:7,1:7))
experimental_design = data.frame(label, condition, replicate)  

data_se <- make_se(data_unique,
                   a_columns, 
                   experimental_design
)

# Filter for proteins that are identified in 2 out of 3 replicates of at least one condition
data_filt <- filter_missval(data_se, 
                            thr=1
)

# Normalize the data
data_norm <- normalize_vsn(data_filt)

# Impute missing data using random draws from a Gaussian distribution centered around a minimal value (for MNAR)
data_imp <- suppressMessages(impute(data_norm, 
                                    fun="MinProb", 
                                    q=0.01)
)

data_diff_manual <- test_diff(data_imp,
                              type="manual",
                              test=c("IFHFD_vs_ALHFD")
)

# Denote significant proteins based on user defined cutoffs
depIF <- add_rejections(data_diff_manual,
                        alpha = 0.05, 
                        lfc = log2(2)
)
res_ALHFD_vs_IFHFD <- get_results(depIF)

[1] 0.1929417


Tested contrasts: IFHFD_vs_ALHFD



In [5]:
res_ALHFD_vs_IFHFD[res_ALHFD_vs_IFHFD[,3] < 0.05,]

## integration transcriptomics and proteomics

In [ ]:
IFfin <- merge(protIF, 
               rnaIF,
               by="GeneName"
)
# calculate a
IFfin["log10Pval_Trans"] = -log10(IFfin$pvalue.y)*sign(IFfin$log2FoldChange.y)
IFfin["log10Pval_Prote"] = -log10(IFfin$pvalue.x)*sign(IFfin$log2FoldChange.x)

cor(IFfin$log10Pval_Trans, 
    IFfin$log10Pval_Prote,
    method="spearman"
)

In [ ]:
protIF1 = protIF[which(protIF$pvalue<cutoff),]

rnaIF1 = rnaIF[which(rnaIF$pvalue<cutoff),]

IFfin1 <- merge(protIF1, 
                rnaIF1,
                by="GeneName"
)

IFfin1["log10Pval_Trans"] = -log10(IFfin1$pvalue.y)*sign(IFfin1$log2FoldChange.y)
IFfin1["log10Pval_Prote"] = -log10(IFfin1$pvalue.x)*sign(IFfin1$log2FoldChange.x)


In [ ]:
# integrated plot

IFgen = c("Baat","Fads1","Fads2","Scd1","Crot","Hacl1","Mlycd","Pmvk","Cyc1","Ndufa3","Ndufa9")
IFfin$significance <- ifelse((IFfin$pvalue.x < 0.05) & (IFfin$pvalue.y < 0.05),
                             "p-value<0.05",
                             "p-value>0.05"
)
IFfin$label <- ifelse(IFfin$GeneName%in%IFgen,
                      "name",
                      ""
)
for(i in 1:nrow(IFfin)){
  if(IFfin$label[i] == "name") IFfin$label[i] = IFfin$GeneName[i]
}

setwd("/Users/viktorian.miok/Documents/consultation/Katarina/Integration_Data/Integration/results/IF/plots/")
pdf("IF_integrated.pdf") 
b <- ggplot(IFfin, aes(x=log10Pval_Trans, y=log10Pval_Prote))
b + geom_point(aes(color=significance), show.legend=FALSE) +
            scale_color_manual(values = c("firebrick", "grey")) +
            geom_hline(yintercept=c(-log10(cutoff), log10(cutoff)), linetype="dashed") +
            geom_vline(xintercept=c(-log10(cutoff), log10(cutoff)), linetype="dashed") +
            xlab("-Log10Pval(transcriptome)*sign(FC)") +
            ylab("-Log10Pval(proteome)*sign(FC)")+
            annotate(geom="text",
                     x=-7.5, 
                     y=6,
                     label=expression(correlaton:rho == 0.19),
                     color="blue") +
            ggtitle("TRF study")+
            geom_text_repel(aes(label=label), 
                            size=5,
                            box.padding=unit(2, "lines"),
                            point.padding=unit(0.1, "lines"),
                            segment.size=0.15) 
dev.off()

In [ ]:
#   tables of significant genes and proteins
colnames(IFfin) = colnames(IFfin1) = c("GeneName","log2FoldChange.protein","pvalue.protein","log2FoldChange.rna",
                                       "pvalue.rna","-Log10Pval(transcriptome)*sign(FC)","-Log10Pval(proteome)*sign(FC)")
IFintegrated_rna = IFfin[IFfin$pvalue.rna < 0.05,]
IFintegrated_protein = IFfin[IFfin$pvalue.protein < 0.05,]
IFintegrated = IFfin1
#setwd("/Users/viktorian.miok/Documents/consultation/Katarina/Integration_Data/Integration/results/IF")
#write.csv(IFintegrated_rna,"IF_integrated_RNA.csv")
#write.csv(IFintegrated_protein,"IF_integrated_protein.csv")
#write.csv(IFintegrated,"IF_integrated_RNA_protein.csv")
